In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [2]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [28]:
url = "https://air-quality-api.open-meteo.com/v1/air-quality"

# in params, can change date for time frame desired (max 3 month window at a time)

params = {
	"latitude": 34.0588,
	"longitude": -118.4439,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide"],
	"start_date": "2025-01-01",
	"end_date": "2025-03-31"
}
responses = openmeteo.weather_api(url, params=params)

In [29]:
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 34.099998474121094°N -118.39999389648438°E
Elevation 103.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


In [30]:
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

In [31]:
hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide

In [32]:
hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


                          date       pm10      pm2_5  carbon_monoxide
0    2025-01-01 00:00:00+00:00  24.200001  23.299999            409.0
1    2025-01-01 01:00:00+00:00  29.700001  29.100000            508.0
2    2025-01-01 02:00:00+00:00  35.400002  34.799999            654.0
3    2025-01-01 03:00:00+00:00  38.200001  37.700001            787.0
4    2025-01-01 04:00:00+00:00  42.799999  42.400002            910.0
...                        ...        ...        ...              ...
2155 2025-03-31 19:00:00+00:00  12.900000  10.700000            227.0
2156 2025-03-31 20:00:00+00:00  11.800000   9.800000            186.0
2157 2025-03-31 21:00:00+00:00  12.300000  10.200000            157.0
2158 2025-03-31 22:00:00+00:00  11.500000   9.500000            152.0
2159 2025-03-31 23:00:00+00:00  10.100000   8.400000            160.0

[2160 rows x 4 columns]


In [33]:
# Change title of csv based on retrieved data
hourly_dataframe.to_csv('winter-2025-aqi.csv', index = False)